#### Illavenil Prabhakaran - Cloud

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt #plotting
import seaborn as sns #good visualizing
import os
import warnings
warnings.filterwarnings('ignore')

from pyspark.context import SparkContext
from pyspark import SparkContext
sc = SparkContext('local')

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/12/08 20:35:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/12/08 20:35:21 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/12/08 20:35:21 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/12/08 20:35:21 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
22/12/08 20:35:21 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.


In [2]:
from pyspark.sql.session import SparkSession
spark = SparkSession.builder.appName('Illavenil').getOrCreate()

In [3]:
path = "TrainingDataset.csv"
val_path = "ValidationDataset.csv"
df = spark.read.option("delimiter", ";").option("header", "true").option("inferSchema","true").csv(path)
val_df = spark.read.option("delimiter", ";").option("header", "true").option("inferSchema","true").csv(val_path)

In [4]:
df.printSchema()

root
 |-- fixed acidity: double (nullable = true)
 |-- volatile acidity: double (nullable = true)
 |-- citric acid: double (nullable = true)
 |-- residual sugar: double (nullable = true)
 |-- chlorides: double (nullable = true)
 |-- free sulfur dioxide: double (nullable = true)
 |-- total sulfur dioxide: double (nullable = true)
 |-- density: double (nullable = true)
 |-- pH: double (nullable = true)
 |-- sulphates: double (nullable = true)
 |-- alcohol: double (nullable = true)
 |-- quality: integer (nullable = true)



In [5]:
from pyspark.sql.functions import isnan, when, count, col

df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+---+---------+-------+-------+
|fixed acidity|volatile acidity|citric acid|residual sugar|chlorides|free sulfur dioxide|total sulfur dioxide|density| pH|sulphates|alcohol|quality|
+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+---+---------+-------+-------+
|            0|               0|          0|             0|        0|                  0|                   0|      0|  0|        0|      0|      0|
+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+---+---------+-------+-------+



In [6]:
feature_col = df.drop('quality').columns
feature_col

['fixed acidity',
 'volatile acidity',
 'citric acid',
 'residual sugar',
 'chlorides',
 'free sulfur dioxide',
 'total sulfur dioxide',
 'density',
 'pH',
 'sulphates',
 'alcohol']

In [7]:
from pyspark.ml.stat import Correlation
from pyspark.ml.feature import VectorAssembler

# convert to vector column first
vector_col = "corr_features"
assembler = VectorAssembler(inputCols=feature_col, outputCol=vector_col)
df_vector = assembler.transform(df).select(vector_col)

# get correlation matrix
matrix = Correlation.corr(df_vector, vector_col).collect()[0][0] 
corr_matrix = matrix.toArray().tolist() 
corr_matrix_df = pd.DataFrame(data=corr_matrix, columns = feature_col, index=feature_col) 
corr_matrix_df .style.background_gradient(cmap='coolwarm').set_precision(2)

22/12/08 20:35:29 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
22/12/08 20:35:29 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
fixed acidity,1.00,-0.25,0.66,0.06,0.11,-0.15,-0.10,0.63,-0.66,0.20,-0.12
volatile acidity,-0.25,1.00,-0.55,0.01,0.07,-0.01,0.09,0.05,0.22,-0.26,-0.22
citric acid,0.66,-0.55,1.00,0.12,0.21,-0.07,0.04,0.33,-0.52,0.32,0.09
residual sugar,0.06,0.01,0.12,1.00,0.08,0.22,0.23,0.36,-0.06,0.00,-0.00
chlorides,0.11,0.07,0.21,0.08,1.00,0.01,0.05,0.21,-0.28,0.41,-0.22
free sulfur dioxide,-0.15,-0.01,-0.07,0.22,0.01,1.00,0.64,0.01,0.06,0.04,-0.08
total sulfur dioxide,-0.10,0.09,0.04,0.23,0.05,0.64,1.00,0.10,-0.08,-0.01,-0.23
density,0.63,0.05,0.33,0.36,0.21,0.01,0.10,1.00,-0.30,0.15,-0.56
pH,-0.66,0.22,-0.52,-0.06,-0.28,0.06,-0.08,-0.30,1.00,-0.16,0.24
sulphates,0.20,-0.26,0.32,0.00,0.41,0.04,-0.01,0.15,-0.16,1.00,0.11


In [8]:
train_assembler = VectorAssembler(inputCols=feature_col,outputCol="features")
df_train = assembler.transform(df)
df_train.show()

+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+--------------------+
|fixed acidity|volatile acidity|citric acid|residual sugar|chlorides|free sulfur dioxide|total sulfur dioxide|density|  pH|sulphates|alcohol|quality|       corr_features|
+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+--------------------+
|          8.9|            0.22|       0.48|           1.8|    0.077|               29.0|                60.0| 0.9968|3.39|     0.53|    9.4|      6|[8.9,0.22,0.48,1....|
|          7.6|            0.39|       0.31|           2.3|    0.082|               23.0|                71.0| 0.9982|3.52|     0.65|    9.7|      5|[7.6,0.39,0.31,2....|
|          7.9|            0.43|       0.21|           1.6|    0.106|               10.0|                37.0| 0.9966|3.17|     0.91|    9.5|    

In [9]:
from pyspark.ml.classification import DecisionTreeClassifier
dt = DecisionTreeClassifier(labelCol = 'quality' , featuresCol = "corr_features")
model = dt.fit(df_train)

In [10]:
model.transform(df_train)

DataFrame[fixed acidity: double, volatile acidity: double, citric acid: double, residual sugar: double, chlorides: double, free sulfur dioxide: double, total sulfur dioxide: double, density: double, pH: double, sulphates: double, alcohol: double, quality: int, corr_features: vector, rawPrediction: vector, probability: vector, prediction: double]

In [11]:
model.write().overwrite().save("./Illa_model")

In [12]:
model.load("./Illa_model")

DecisionTreeClassificationModel: uid=DecisionTreeClassifier_756b03b07f00, depth=5, numNodes=43, numClasses=9, numFeatures=11

In [13]:
display(model)

DecisionTreeClassificationModel: uid=DecisionTreeClassifier_756b03b07f00, depth=5, numNodes=43, numClasses=9, numFeatures=11

In [14]:
validation_assembler = VectorAssembler(inputCols=feature_col,outputCol="features")
df_validation = assembler.transform(val_df)
df_validation.show()

+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+--------------------+
|fixed acidity|volatile acidity|citric acid|residual sugar|chlorides|free sulfur dioxide|total sulfur dioxide|density|  pH|sulphates|alcohol|quality|       corr_features|
+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+--------------------+
|          7.4|             0.7|        0.0|           1.9|    0.076|                 11|                  34| 0.9978|3.51|     0.56|    9.4|      5|[7.4,0.7,0.0,1.9,...|
|          7.8|            0.88|        0.0|           2.6|    0.098|                 25|                  67| 0.9968| 3.2|     0.68|    9.8|      5|[7.8,0.88,0.0,2.6...|
|          7.8|            0.76|       0.04|           2.3|    0.092|                 15|                  54|  0.997|3.26|     0.65|    9.8|    

In [15]:
predictions = model.transform(df_validation)
predictions.select('quality', 'rawPrediction', 'prediction', 'probability').show()

+-------+--------------------+----------+--------------------+
|quality|       rawPrediction|prediction|         probability|
+-------+--------------------+----------+--------------------+
|      5|[0.0,0.0,0.0,4.0,...|       5.0|[0.0,0.0,0.0,0.00...|
|      5|[0.0,0.0,0.0,4.0,...|       5.0|[0.0,0.0,0.0,0.00...|
|      5|[0.0,0.0,0.0,4.0,...|       5.0|[0.0,0.0,0.0,0.00...|
|      6|[0.0,0.0,0.0,1.0,...|       6.0|[0.0,0.0,0.0,0.01...|
|      5|[0.0,0.0,0.0,4.0,...|       5.0|[0.0,0.0,0.0,0.00...|
|      5|[0.0,0.0,0.0,4.0,...|       5.0|[0.0,0.0,0.0,0.00...|
|      5|[0.0,0.0,0.0,4.0,...|       5.0|[0.0,0.0,0.0,0.00...|
|      7|[0.0,0.0,0.0,4.0,...|       5.0|[0.0,0.0,0.0,0.00...|
|      7|[0.0,0.0,0.0,4.0,...|       5.0|[0.0,0.0,0.0,0.00...|
|      5|[0.0,0.0,0.0,0.0,...|       5.0|[0.0,0.0,0.0,0.0,...|
|      5|[0.0,0.0,0.0,4.0,...|       5.0|[0.0,0.0,0.0,0.00...|
|      5|[0.0,0.0,0.0,0.0,...|       5.0|[0.0,0.0,0.0,0.0,...|
|      5|[0.0,0.0,0.0,4.0,...|       5.0|[0.0,0.0,0.0,0

In [20]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(labelCol="quality", predictionCol="prediction", metricName='accuracy')
accuracy = evaluator.evaluate(predictions)
print("Accuracy = %s" % (accuracy))
print("Test Error = %s" % (1.0 - accuracy))

Accuracy = 0.48125
Test Error = 0.51875


In [17]:
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.sql.types import FloatType
import pyspark.sql.functions as F

preds_and_labels = predictions.select(['prediction','quality']).withColumn('quality', F.col('quality').cast(FloatType())).orderBy('prediction')
preds_and_labels = preds_and_labels.select(['prediction','quality'])
metrics = MulticlassMetrics(preds_and_labels.rdd.map(tuple))
print(metrics.confusionMatrix().toArray())

[[ 0.  0.  0.  1.  0.  0.]
 [ 0.  0.  2.  0.  0.  0.]
 [ 0.  2. 44. 20.  0.  0.]
 [ 0.  1. 32. 20. 12.  0.]
 [ 0.  0.  3.  6. 13.  0.]
 [ 0.  0.  0.  0.  4.  0.]]


In [21]:
#f1_score
my_mc_lr = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='quality', metricName='f1')
print("F1 :", my_mc_lr.evaluate(predictions))

F1 : 0.462126100440176
